<a href="https://colab.research.google.com/github/srivatsan88/Tensorflow2.0_Samples/blob/master/Tensorflow_save_load_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q tf-nightly-2.0-preview
!pip install -q h5py pyyaml 

In [2]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf

tf.__version__

'2.0.0-dev20190228'

In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [5]:
def create_model():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
  return model


model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)
model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 992/1000 [============================>.] - ETA: 0s - loss: 1.1591 - accuracy: 0.6613
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 584us/sample - loss: 1.1536 - accuracy: 0.6630 - val_loss: 0.7890 - val_accuracy: 0.7420
Epoch 2/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.4340 - accuracy: 0.8760
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 347us/sample - loss: 0.4324 - accuracy: 0.8760 - val_loss: 0.5212 - val_accuracy: 0.8450
Epoch 3/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.2929 - accuracy: 0.9275
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 333us/sample - loss: 0.2899 - accuracy: 0.9300 - val_loss: 0.4625 - val_accuracy: 0.8640
Epoch 4/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.1875 - accuracy: 0.9635
Epoch

In [7]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [8]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 147us/sample - loss: 2.3562 - accuracy: 0.1120
Untrained model, accuracy: 11.20%


In [9]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 90us/sample - loss: 0.4280 - accuracy: 0.8610
Restored model, accuracy: 86.10%
